In [1]:
import os.path

from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *
from fastaudio.ci import skip_if_ci
import random

C:\Program Files\Python39\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Program Files\Python39\lib\site-packages\torchaudio\backend\utils.py:46: UserWarning: "torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. Please remove the use of flag.
  warnings.warn(


In [2]:
cfg = AudioConfig.Voice()
a2s = AudioToSpec.from_cfg(cfg)

In [19]:
auds = DataBlock(blocks = (AudioBlock),
                 get_x = ColReader("filename"),
                 splitter = RandomSplitter(seed = 42),
                 item_tfms = [ResizeSignal(3000), a2s, MaskFreq(), MaskTime(), SignalShifter(), SignalCutout(), ChangeVolume()],
                 batch_tfms = None)

In [20]:
fnames = get_files("data/TIL_test_normalized/")
df = pd.DataFrame()
df['filename'] = [str(fn) for fn in fnames]
#df['category'] = [str(fn).split(os.path.sep)[-1].split('.')[0] for fn in fnames]

In [21]:
dbunch = auds.dataloaders(df, bs=64)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


In [40]:
# load trained fastai model
learn = cnn_learner(dbunch,
            resnet50,
            n_out = 5,
            lr=1e-3, opt_func = ranger,
            n_in=1,  # <- This is the only audio specific modification here
            loss_func=CrossEntropyLossFlat(),
            metrics=[accuracy, F1Score(average='weighted')])

learn.load('til_data_only_resnet50_9090')

TypeError: 'int' object is not callable

In [ ]:
pred_raw = []
filenames = []
import glob
for file in glob.glob('data/TIL_test_normalized/*.wav'):
    filenames.append(file)
    pred_raw.append(learn.predict(file))

In [52]:
preds = [p[-1] for p in pred_raw]
filenames = [f.split(os.path.sep)[-1] for f in filenames]

In [53]:
from numpy import argmax
preds2 = [argmax(l) for l in preds]
class_index = {0: 'angry', 1: 'fear', 2: 'happy',3: 'neutral', 4: 'sad'}
preds2 = [class_index[int(l.numpy())] for l in preds2]
preds2

['angry',
 'angry',
 'fear',
 'sad',
 'sad',
 'angry',
 'sad',
 'neutral',
 'happy',
 'fear',
 'happy',
 'fear',
 'sad',
 'neutral',
 'sad',
 'angry',
 'fear',
 'angry',
 'happy',
 'sad',
 'happy',
 'happy',
 'happy',
 'sad',
 'sad',
 'happy',
 'neutral',
 'happy',
 'fear',
 'sad',
 'sad',
 'happy',
 'neutral',
 'happy',
 'angry',
 'sad',
 'happy',
 'sad',
 'happy',
 'angry',
 'happy',
 'angry',
 'happy',
 'fear',
 'angry',
 'sad',
 'sad',
 'angry',
 'angry',
 'neutral',
 'angry',
 'angry',
 'angry',
 'sad',
 'angry',
 'neutral',
 'angry',
 'angry',
 'neutral',
 'angry',
 'happy',
 'sad',
 'happy',
 'angry',
 'sad',
 'happy',
 'happy',
 'sad',
 'happy',
 'sad',
 'angry',
 'sad',
 'sad',
 'neutral',
 'neutral',
 'happy',
 'sad',
 'neutral',
 'sad',
 'fear',
 'sad',
 'sad',
 'sad',
 'angry',
 'angry',
 'angry',
 'neutral',
 'angry',
 'happy',
 'fear',
 'angry',
 'happy',
 'angry',
 'happy',
 'angry',
 'angry',
 'happy',
 'sad',
 'angry',
 'happy',
 'sad',
 'happy',
 'fear',
 'sad',
 'hap

In [54]:
pred_df = pd.DataFrame({'filename': filenames, 'pred': preds2})
# sort pred df by filename
pred_df.sort_values(by=['filename'], inplace=True)
pred_df.to_csv('submission_fastai.csv', index=False, header=None)